<h3> A Neural Network Implementation for Pattern Classification using Hebb's rule </h3>

<b>Reading input images </b>

In [3]:
import numpy as np
import os

In [88]:
filePath = "./pattern_data/"

In [89]:
def read_file(fileName):
    file = os.path.join(filePath,fileName)
    print('Quering file {}'.format(file))
    
    pattern, labels,pixel =[],[],[];
    with open(file,"r") as f:
        lines = f.read().splitlines()
        for line in lines:
            for ch in line:
                if ch == "#" or ch == ".":
                    pixel.append(ch)
                elif ch ==",":
                    pattern.append(pixel)
                    pixel=[]
                else:
                    labels.append(ch)
            
    print('..Completed processing file..."')
    return pattern, labels
    
    
    
    

In [90]:
train_patterns,train_labels = read_file(fileName="training_data.txt")

Quering file ./pattern_data/training_data.txt
..Completed processing file..."


In [91]:
test_patterns,test_labels = read_file(fileName="test_data.txt")


Quering file ./pattern_data/test_data.txt
..Completed processing file..."


In [92]:
def convertPatternToBipolarVectors(patterns):
    input_vector,image = [],[];
    for pattern in patterns:
        for  ch in pattern:
            if ch == "#":
                image.append(1)
            elif ch == ".":
                image.append(-1)
                
        input_vector.append(image)
        image=[]
        
    return input_vector

def convertLabelToBipolar(labels):
    output_vector =[];
    
    for ch in labels:
        if ch == "C":
            output_vector.append(1)
        elif ch=="X":
            output_vector.append(-1)
        else:
            output_vector.append(0)
            
    return output_vector
            

In [93]:
bipolar_train = convertPatternToBipolarVectors(train_patterns)
bipolar_train_label = convertLabelToBipolar(train_labels)

In [94]:
bipolar_test = convertPatternToBipolarVectors(test_patterns)

In [54]:
bipolar_test
                                      

[[-1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1]]

In [95]:
def flipPixelByKElements(pattern,numOfElements):
    L = len(pattern)  
    image = pattern.copy()
    listOfPatterns = []
    corrupted_image =list()
    for i in range (0,L-1):
        image = pattern.copy()
        k = numOfElements
        c=i
        while k > 0:
                if image[c] == 1:
                    image[c] = -1
                elif image[c] == -1:
                    image[c] = 1
                if c >= L-1:
                    c=0
                
                c = c+1
                k = k-1
        corrupted_image = image
        listOfPatterns.append(corrupted_image)
     
    return listOfPatterns         
                
                
            
        
    

In [96]:
bipolar_testset = flipPixelByKElements(bipolar_test[0],2)

In [14]:
[-1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1]

[['B', 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1]]

In [ ]:

[[-1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1]]

In [97]:
def convertTrainToNumpyArray(data,labels):
    images = np.array(data)
    dim = labels[0]
    t = np.array(labels).reshape(-1,dim)
    return images,t

def convertTestToNumpyArray(data):
    images = np.array(data)
    return images


In [98]:
patterns_train,t = convertTrainToNumpyArray(bipolar_train,bipolar_train_label)

In [99]:
print('Training Images dimensions',patterns_train.shape)
print('Training Labels dimensions',t.shape)

Training Images dimensions (2, 15)
Training Labels dimensions (2, 1)


In [100]:
patterns_test = convertTestToNumpyArray(bipolar_testset)

In [101]:
print('Testing Images dimensions',patterns_test.shape)

     

Testing Images dimensions (14, 15)


<b>Define the dimensions for a two layered  Neural Network Perceptron Model</b>

In [102]:

dimensions = [patterns_train.shape[1],t.shape[1]]
parameters = {}


<b>Define the Neural Network</b>

In [103]:
class NeuralNetwork:
    def __init__(self,dimensions):
        self.parameters = parameters
        
        print('...Initializing Parameters for the model...')
        parameters["W"] = np.zeros([dimensions[0],dimensions[1]])
        parameters["b"] = np.zeros([dimensions[1],1])
        
        #print('Weight Matrix defined as {}'.format(parameters["W"].shape))
        #print('Bias Vector defined as {}'.format(parameters["b"].shape))
        
    def affineForward(self,X,W,b):
        print('.....Staring affineForward.....')
        yin = np.dot(X,W) + b      
        print(yin)
        return yin
    
    def feedForwardPropagation(self,X):
        print('....Start FeedForwardPropagation....')
        
        W = self.parameters["W"]
        b = self.parameters["b"]
        yin = self.affineForward(X,W,b)
        
        print ('Shape of output {}'.format(yin.shape))
        return yin
    
    def backwardPropagation(self,t,S):
        print('....Starting backpropagation.....')
        dw = np.dot(S.T,t)
        db = np.sum(t,axis=0).reshape(1,-1)
        print('dw')
        print(dw)
        print('db')
        print(db)
        print('Delta W:{}'.format(dw.shape))
        print('Delta b:{}'.format(db.shape))
        return dw,db
    
    def updateParameters(self,dw,db):
        print('....Updating Weight and bias.....')
        parameters = self.parameters
        
        parameters["W"] = np.add(parameters["W"],dw)
        parameters["b"] = np.add(parameters["b"],db)
        
        print("W")
        print(parameters["W"])
        print("b")
        print(parameters["b"])
        
        
    def train(self,X,t,numOfIterations):
        S = X
        y = t
        print('......Training Data Model......')
        for i in range(1,numOfIterations):
            print('Starting iteration {}*********'.format(i))
            yin = self.feedForwardPropagation(X)
            dw,db = self.backwardPropagation(y,S)
            self.updateParameters(dw,db)
        
        print('...Training ends here....')
        
     
    def predict(self,X):
        t = self.feedForwardPropagation(X)
        return t
    

<b> Train Model </b>

In [104]:
model = NeuralNetwork(dimensions)


...Initializing Parameters for the model...


In [105]:
model.train(patterns_train,t,3)

......Training Data Model......
Starting iteration 1*********
....Start FeedForwardPropagation....
.....Staring affineForward.....
[[0.]
 [0.]]
Shape of output (2, 1)
....Starting backpropagation.....
dw
[[-2]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [-2]
 [ 2]
 [-2]
 [ 0]
 [ 0]
 [ 0]
 [-2]
 [-2]
 [ 2]
 [ 0]]
db
[[0]]
Delta W:(15, 1)
Delta b:(1, 1)
....Updating Weight and bias.....
W
[[-2.]
 [ 2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-2.]
 [ 2.]
 [-2.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-2.]
 [-2.]
 [ 2.]
 [ 0.]]
b
[[0.]]
Starting iteration 2*********
....Start FeedForwardPropagation....
.....Staring affineForward.....
[[ 16.]
 [-16.]]
Shape of output (2, 1)
....Starting backpropagation.....
dw
[[-2]
 [ 2]
 [ 0]
 [ 0]
 [ 0]
 [-2]
 [ 2]
 [-2]
 [ 0]
 [ 0]
 [ 0]
 [-2]
 [-2]
 [ 2]
 [ 0]]
db
[[0]]
Delta W:(15, 1)
Delta b:(1, 1)
....Updating Weight and bias.....
W
[[-4.]
 [ 4.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-4.]
 [ 4.]
 [-4.]
 [ 0.]
 [ 0.]
 [ 0.]
 [-4.]
 [-4.]
 [ 4.]
 [ 0.]]
b
[[0.]]
...Training ends here....


<b>Make Predictions </b>

In [106]:
t_pred = model.predict(patterns_test)

....Start FeedForwardPropagation....
.....Staring affineForward.....
[[16.]
 [24.]
 [32.]
 [32.]
 [24.]
 [16.]
 [16.]
 [24.]
 [32.]
 [32.]
 [24.]
 [16.]
 [16.]
 [24.]]
Shape of output (14, 1)


In [ ]:
t_pred

In [ ]:
t_pred = list(t_pred)

In [ ]:
output = [ v for v in t_pred if v > 0]

In [ ]:
output